In [1]:
# 1) eliminar protobuf problemático
!pip uninstall -y protobuf

# 2) instalar una versión compatible (3.20.x suele resolverlo) y mediapipe
!pip install protobuf==3.20.3 mediapipe -q


Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 85.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
import glob
import urllib.request
import time
import tracemalloc
import tempfile
import joblib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random

# Set Global Seeds for Reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Configure Plotting Style
sns.set_style("whitegrid")
plt.rcParams.update({'font.size': 12})
# Use Agg backend to save plots without display
plt.switch_backend('Agg')

2026-01-04 21:13:48.610537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767561229.069506      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767561229.199987      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
### 1. MediaPipe Setup (Updated for MediaPipe Tasks API)

def download_model(url, filename):
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, filename)
        print("Done.")

download_model('https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_full/float16/1/pose_landmarker_full.task', 'pose_landmarker_full.task')
download_model('https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task', 'hand_landmarker.task')

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

pose_options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='pose_landmarker_full.task'),
    running_mode=VisionRunningMode.IMAGE,
    min_pose_detection_confidence=0.5,
    min_pose_presence_confidence=0.5,
    min_tracking_confidence=0.5,
    output_segmentation_masks=False
)
pose_landmarker = PoseLandmarker.create_from_options(pose_options)

hand_options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=2,
    min_hand_detection_confidence=0.5,
    min_hand_presence_confidence=0.5,
    min_tracking_confidence=0.5
)
hand_landmarker = HandLandmarker.create_from_options(hand_options)

POSE_INDICES = [11, 12, 13, 14, 15, 16]

def get_landmarks(image):
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    
    results_pose = pose_landmarker.detect(mp_image)
    results_hands = hand_landmarker.detect(mp_image)
    
    if not results_pose.pose_landmarks:
        return None
        
    pl = results_pose.pose_landmarks[0]

    required_indices = [11, 12, 13, 14]
    for idx in required_indices:
        if pl[idx].visibility < 0.25:
            return None
    
    left_shoulder = np.array([pl[11].x, pl[11].y])
    right_shoulder = np.array([pl[12].x, pl[12].y])
    
    shoulder_center = (left_shoulder + right_shoulder) / 2.0
    shoulder_width = np.linalg.norm(left_shoulder - right_shoulder)
    
    if shoulder_width < 1e-6: 
        shoulder_width = 1.0
    
    current_frame_points = []
    
    safe_wrists = {}
    
    for idx in POSE_INDICES:
        target_landmark = pl[idx]
        p = np.array([target_landmark.x, target_landmark.y])
        
        if idx == 15: # Muñeca Izq
            if target_landmark.visibility < 0.25:
                elbow = pl[13]
                p = np.array([elbow.x, elbow.y])
            safe_wrists['Left'] = p
        
        elif idx == 16: # Muñeca Der
            if target_landmark.visibility < 0.25:
                elbow = pl[14]
                p = np.array([elbow.x, elbow.y])
            safe_wrists['Right'] = p

        norm_p = (p - shoulder_center) / shoulder_width
        current_frame_points.extend(norm_p)
         
    left_hand_raw = None
    right_hand_raw = None
    
    if results_hands.hand_landmarks:
        for idx, hand_lm_list in enumerate(results_hands.hand_landmarks):
            if idx < len(results_hands.handedness):
                label = results_hands.handedness[idx][0].category_name
            else:
                label = 'Unknown'
            
            h_points = []
            for hl in hand_lm_list:
                h_points.append([hl.x, hl.y])
            
            if label == 'Left':
                left_hand_raw = np.array(h_points)
            else:
                right_hand_raw = np.array(h_points)
                
    # Rellenar manos faltantes con posición de la muñeca
    if left_hand_raw is None:
        ref_point = safe_wrists.get('Left', np.zeros(2))
        left_hand_raw = np.tile(ref_point, (21, 1))
        
    if right_hand_raw is None:
        ref_point = safe_wrists.get('Right', np.zeros(2))
        right_hand_raw = np.tile(ref_point, (21, 1))

    # Normalizar manos
    for point in left_hand_raw:
        norm_p = (point - shoulder_center) / shoulder_width
        current_frame_points.extend(norm_p)
        
    for point in right_hand_raw:
        norm_p = (point - shoulder_center) / shoulder_width
        current_frame_points.extend(norm_p)
    
    return current_frame_points

def interpolate_sequence(sequence, target_len=40):
    sequence = np.array(sequence)
    seq_len = len(sequence)
    
    if seq_len == 0:
        return np.zeros((target_len, 96))
    
    if seq_len == 1:
        return np.repeat(sequence, target_len, axis=0)
        
    current_x = np.linspace(0, 1, seq_len)
    target_x = np.linspace(0, 1, target_len)
    
    f = interp1d(current_x, sequence, axis=0, kind='linear', fill_value="extrapolate")
    interpolated_sequence = f(target_x)
    
    return interpolated_sequence

Done.
Done.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1767561250.523626     128 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767561250.575218     128 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767561250.615007     132 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767561250.636649     132 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
### 3. Data Loading
DATA_PATH = "/kaggle/input/lsa-64-dinamico/all/*.mp4"
files = glob.glob(DATA_PATH)
print(f"Found {len(files)} files.")

X_seq_list = [] # Para almacenar secuencias (N, TimeSteps, Features)
y_list = []
skipped_count = 0

for i, file_path in enumerate(files):
    filename = os.path.basename(file_path)
    label = filename.split('_')[0]
    
    # Filtrar solo tus clases deseadas si es necesario, o usar todas
    # allowed_labels = ["001", "002"] # Ejemplo
    # if label not in allowed_labels: continue

    if i % 10 == 0: print(f"Processing {i}/{len(files)}: {filename}")
    
    cap = cv2.VideoCapture(file_path)
    video_landmarks = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        points = get_landmarks(image)
        if points is not None:
             video_landmarks.append(points)
             
    cap.release()
    
    if len(video_landmarks) > 0:
        video_landmarks_interp = interpolate_sequence(video_landmarks, target_len=40)
        X_seq_list.append(video_landmarks_interp) # Shape: (40, 96)
        y_list.append(label)
    else:
        print(f"⚠️ VIDEO DESCARTADO: {filename}")
        skipped_count += 1

X_seq = np.array(X_seq_list) # Data secuencial: (N, 40, 96)
y = np.array(y_list)

if len(X_seq) == 0:
    print("No data loaded. Change DATA_PATH or check files.")
    exit()

# Flatten data for Traditional ML (N, 40*96) -> (N, 3840)
X_flat = X_seq.reshape(X_seq.shape[0], -1)

# Encode Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_

print(f"\nProcesamiento finalizado.")
print(f"Total Videos: {len(X_seq)}")
print(f"Data Secuencial Shape (DL): {X_seq.shape}")
print(f"Data Plana Shape (ML): {X_flat.shape}")
print(f"Clases detectadas: {num_classes}")

Found 3200 files.
Processing 0/3200: 006_006_005.mp4


W0000 00:00:1767561250.987221     126 landmark_projection_calculator.cc:78] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Processing 10/3200: 027_002_003.mp4
Processing 20/3200: 064_003_004.mp4
Processing 30/3200: 015_005_003.mp4
Processing 40/3200: 002_002_001.mp4
Processing 50/3200: 023_006_002.mp4
Processing 60/3200: 034_009_002.mp4
Processing 70/3200: 045_006_004.mp4
Processing 80/3200: 018_001_005.mp4
Processing 90/3200: 032_002_005.mp4
Processing 100/3200: 053_001_004.mp4
Processing 110/3200: 001_004_001.mp4
Processing 120/3200: 061_010_001.mp4
Processing 130/3200: 047_007_004.mp4
Processing 140/3200: 055_006_001.mp4
Processing 150/3200: 025_006_001.mp4
Processing 160/3200: 034_010_005.mp4
Processing 170/3200: 026_007_003.mp4
Processing 180/3200: 035_001_003.mp4
Processing 190/3200: 058_005_002.mp4
Processing 200/3200: 001_006_002.mp4
Processing 210/3200: 050_009_001.mp4
Processing 220/3200: 046_002_003.mp4
Processing 230/3200: 047_001_004.mp4
Processing 240/3200: 057_009_005.mp4
Processing 250/3200: 064_007_001.mp4
Processing 260/3200: 045_002_004.mp4
Processing 270/3200: 039_005_001.mp4
Processing

In [5]:
### 4. Classification & Metrics

# Split Data
X_train_flat, X_test_flat, y_train_flat, y_test_flat = train_test_split(X_flat, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(X_seq, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Data Scaling (Standardization)
# 1. Flat Data (ML)
scaler_flat = StandardScaler()
X_train_flat = scaler_flat.fit_transform(X_train_flat)
X_test_flat = scaler_flat.transform(X_test_flat)

# 2. Sequential Data (DL)
# Scaler works on 2D, so we reshape (N, T, F) -> (N*T, F) then back
N_train, T, F = X_train_seq.shape
N_test = X_test_seq.shape[0]

scaler_seq = StandardScaler()
X_train_seq_2d = X_train_seq.reshape(-1, F)
X_test_seq_2d = X_test_seq.reshape(-1, F)

X_train_seq_2d = scaler_seq.fit_transform(X_train_seq_2d)
X_test_seq_2d = scaler_seq.transform(X_test_seq_2d)

X_train_seq = X_train_seq_2d.reshape(N_train, T, F)
X_test_seq = X_test_seq_2d.reshape(N_test, T, F)

results_list = []

def get_model_size_mb(model, model_name, is_keras=False):
    try:
            if is_keras:
                filepath = f"/kaggle/working/entrenamiento_{model_name}.h5"
                model.save(filepath)
            else:
                filepath = f"/kaggle/working/entrenamiento_{model_name}.joblib"
                joblib.dump(model, filepath)
            size = os.path.getsize(filepath)
            return size / (1024 * 1024) # MB
    except Exception as e:
        print(f"Error calculating size for {model_name}: {e}")
        return 0

def plot_confusion_matrix_custom(y_true, y_pred, model_name, labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=False, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f'/kaggle/working/confusion_matrix_{model_name}.png', dpi=300)
    plt.close()

def evaluate_model(model, X_train, y_train, X_test, y_test, model_name, is_keras=False):
    print(f"\nEvaluating {model_name}...")
    
    # 1. Training Resources
    tracemalloc.start()
    start_time_train = time.time()
    
    if is_keras:
        # EarlyStopping para DL
        callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=40, batch_size=16, 
                  validation_split=0.1, callbacks=[callback], verbose=1)
    else:
        model.fit(X_train, y_train)
        
    train_time = time.time() - start_time_train
    current, peak = tracemalloc.get_traced_memory()
    peak_ram_mb = peak / (1024 * 1024)
    tracemalloc.stop()
    
    # 2. Model Size
    model_size_mb = get_model_size_mb(model, model_name, is_keras)
    
    # 3. Inference Time
    start_time_inf = time.time()
    if is_keras:
        y_pred_probs = model.predict(X_test, verbose=0)
        y_pred = np.argmax(y_pred_probs, axis=1)
    else:
        y_pred = model.predict(X_test)
    inf_time_total = time.time() - start_time_inf
    inf_time_per_sample_ms = (inf_time_total / len(X_test)) * 1000
    
    # 4. Metrics
    acc = accuracy_score(y_test, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted', zero_division=0)
    
    # 5. Confusion Matrix
    plot_confusion_matrix_custom(y_test, y_pred, model_name, class_names)
    
    print(f"  Accuracy: {acc:.4f}")
    print(f"  Training Time: {train_time:.2f}s")
    
    return {
        "Model": model_name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1 Score": f1,
        "Training Time (s)": train_time,
        "Inference Time per Sample (ms)": inf_time_per_sample_ms,
        "Model Size (MB)": model_size_mb,
        "Peak RAM (MB)": peak_ram_mb
    }

# --- A. Traditional ML ---
ml_classifiers = {
    "ExtraTrees": ExtraTreesClassifier(n_estimators=100, random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='rbf', random_state=42),
    "XGBoost": xgb.XGBClassifier(eval_metric='mlogloss', random_state=42)
}

for name, clf in ml_classifiers.items():
    res = evaluate_model(clf, X_train_flat, y_train_flat, X_test_flat, y_test_flat, name, is_keras=False)
    results_list.append(res)

# --- B. Deep Learning ---
def build_lstm(input_shape, num_classes):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        # Capa 1: Extracción de secuencias complejas
        layers.LSTM(128, return_sequences=True),
        layers.Dropout(0.3),
        # Capa 2: Compresión y clasificación
        layers.LSTM(64),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def build_gru(input_shape, num_classes):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.GRU(128, return_sequences=True),
        layers.Dropout(0.3),
        layers.GRU(64),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim
        self.projection = layers.Dense(output_dim)

    def call(self, inputs):
        x = self.projection(inputs)
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return x + embedded_positions

    def get_config(self):
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "output_dim": self.output_dim,
        })
        return config

def build_transformer(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    
    # --- Positional Embedding ---
    d_model = 128
    x = PositionalEmbedding(sequence_length=input_shape[0], output_dim=d_model)(inputs)
    
    # --- Transformer Block ---
    # 1. Multi-Head Attention
    x_norm = layers.LayerNormalization(epsilon=1e-6)(x)
    attention_output = layers.MultiHeadAttention(key_dim=d_model, num_heads=4, dropout=0.1)(x_norm, x_norm)
    x = layers.Add()([x, attention_output]) # Residual connection
    
    # 2. Feed Forward Network (FFN)
    y = layers.LayerNormalization(epsilon=1e-6)(x)
    y = layers.Conv1D(filters=d_model*2, kernel_size=1, activation='relu')(y) 
    y = layers.Dropout(0.1)(y)
    y = layers.Conv1D(filters=d_model, kernel_size=1)(y) 
    x = layers.Add()([x, y]) # Residual connection
    
    # --- Classification Head ---
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

dl_constructors = {
    "LSTM": build_lstm,
    "GRU": build_gru,
    "Transformer": build_transformer
}

for name, constructor in dl_constructors.items():
    model = constructor((40, 96), num_classes)
    res = evaluate_model(model, X_train_seq, y_train_seq, X_test_seq, y_test_seq, name, is_keras=True)
    results_list.append(res)


# --- C. Export & Analysis ---
df_results = pd.DataFrame(results_list)
df_results.to_csv("/kaggle/working/metrics_results.csv", index=False)
print("\nMetrics exported to metrics_results.csv")
print(df_results)

# --- D. Comparative Plots ---

# 1. Training & Inference Time
df_melt_time = df_results.melt(id_vars="Model", value_vars=["Training Time (s)", "Inference Time per Sample (ms)"], var_name="Metric", value_name="Time")
plt.figure(figsize=(12, 6))
sns.barplot(data=df_melt_time, x="Model", y="Time", hue="Metric")
plt.title("Training vs Inference Time")
plt.yscale("log") # Log scale usually better for these comparisons
plt.ylabel("Time (Log Scale)")
plt.tight_layout()
plt.savefig("/kaggle/working/plot_time_comparison.png", dpi=300)
plt.close()

# 2. Disk Size & Peak RAM
df_melt_size = df_results.melt(id_vars="Model", value_vars=["Model Size (MB)", "Peak RAM (MB)"], var_name="Metric", value_name="Size (MB)")
plt.figure(figsize=(12, 6))
sns.barplot(data=df_melt_size, x="Model", y="Size (MB)", hue="Metric")
plt.title("Model Size vs Peak RAM")
plt.tight_layout()
plt.savefig("/kaggle/working/plot_size_ram_comparison.png", dpi=300)
plt.close()

# 3. Accuracy vs Inference Time (Scatter)
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_results, x="Inference Time per Sample (ms)", y="Accuracy", s=300, hue="Model", style="Model")
for i in range(df_results.shape[0]):
    plt.text(df_results["Inference Time per Sample (ms)"][i], df_results["Accuracy"][i]+0.005, 
             df_results["Model"][i], fontsize=10, ha='center')
plt.title("Accuracy vs Inference Time")
plt.grid(True)
plt.tight_layout()
plt.savefig("/kaggle/working/plot_accuracy_vs_inference.png", dpi=300)
plt.close()

print("\nAll plots saved as PNG images.")


Evaluating ExtraTrees...
  Accuracy: 0.9703
  Training Time: 3.32s

Evaluating RandomForest...
  Accuracy: 0.9625
  Training Time: 21.96s

Evaluating SVM...
  Accuracy: 0.9219
  Training Time: 7.76s

Evaluating XGBoost...
  Accuracy: 0.9359
  Training Time: 287.41s


I0000 00:00:1767594522.124991      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1767594522.125686      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5



Evaluating LSTM...
Epoch 1/40


I0000 00:00:1767594533.767029   12964 cuda_dnn.cc:529] Loaded cuDNN version 90300


144/144 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.1944 - loss: 3.5824 - val_accuracy: 0.4883 - val_loss: 2.3660
Epoch 2/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5448 - loss: 2.0094 - val_accuracy: 0.7031 - val_loss: 1.3505
Epoch 3/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6970 - loss: 1.2481 - val_accuracy: 0.7969 - val_loss: 0.8809
Epoch 4/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7859 - loss: 0.8663 - val_accuracy: 0.8281 - val_loss: 0.7053
Epoch 5/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8300 - loss: 0.6614 - val_accuracy: 0.8438 - val_loss: 0.6258
Epoch 6/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8517 - loss: 0.5843 - val_accuracy: 0.8828 - val_loss: 0.4656
Epoch 7/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8776 - loss: 0.4586 - val_accuracy: 0.8945 - val_loss: 0.3599
Epoch 8/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9153 - loss: 0.3493 - val_accuracy: 0.925

...

I0000 00:00:1767594646.870629   12964 service.cc:148] XLA service 0x160494c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767594646.871989   12964 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1767594646.872011   12964 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 27/144 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0829 - loss: 4.2601   

I0000 00:00:1767594653.279212   12964 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


144/144 ━━━━━━━━━━━━━━━━━━━━ 24s 21ms/step - accuracy: 0.2314 - loss: 3.2202 - val_accuracy: 0.6758 - val_loss: 1.1241
Epoch 2/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6687 - loss: 1.1109 - val_accuracy: 0.8281 - val_loss: 0.5721
Epoch 3/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8047 - loss: 0.6227 - val_accuracy: 0.8867 - val_loss: 0.3535
Epoch 4/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8655 - loss: 0.4301 - val_accuracy: 0.9062 - val_loss: 0.2829
Epoch 5/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8999 - loss: 0.2897 - val_accuracy: 0.8750 - val_loss: 0.4224
Epoch 6/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8893 - loss: 0.3251 - val_accuracy: 0.9609 - val_loss: 0.1414
Epoch 7/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9278 - loss: 0.1961 - val_accuracy: 0.9531 - val_loss: 0.1988
Epoch 8/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9216 - loss: 0.2283 - val_accuracy: 0.9375 - va

  Accuracy: 0.9469
  Training Time: 35.00s

Metrics exported to metrics_results.csv
          Model  Accuracy  Precision    Recall  F1 Score  Training Time (s)  \
0    ExtraTrees  0.970313   0.972990  0.970313  0.970272           3.317426   
1  RandomForest  0.962500   0.968386  0.962500  0.962184          21.963737   
2           SVM  0.921875   0.933647  0.921875  0.920125           7.756842   
3       XGBoost  0.935937   0.940250  0.935937  0.935365         287.410970   
4          LSTM  0.960938   0.968711  0.960938  0.959891          47.790968   
5           GRU  0.985938   0.987453  0.985938  0.985680          55.687465   
6   Transformer  0.946875   0.952556  0.946875  0.945312          35.003115   

   Inference Time per Sample (ms)  Model Size (MB)  Peak RAM (MB)  
0                        0.074816        56.132226      41.321878  
1                        0.069121        27.803079      41.317204  
2                        8.663294        66.292380      82.655665  
3          